### Bentheimer Sandstoneの物性値  
参考：V.Voronina, L. Pel and K. Kopinga: The influence of osmotic pressure on poulticing treatments for heritage objects, Material and Structures, vol.46, pp221-231, 2013  


### 関数の命名方法  
・物性値名のみ：関数の変換が無く、材料固有の物性値の場合  
・get+物性値名：他の物質量に依存する物性値の場合

## 1. 材料の熱物性値

$\psi$：空隙率(porosity)[-]  

$\rho$：材料の密度(density)[kg/m3]

$C$：材料の比熱(specific heat)[J/(kg・K)]  

$c\rho$：材料の熱容量(heat capacity)[J/(m3・K)]  

## 2. 材料の水分特性

$\phi$：含水率[m3/m3]

$\frac{\partial \phi}{\partial \mu}$：含水率の水分化学ポテンシャル微分[m3/m3(J/kg)]

$\mu$：水分化学ポテンシャル[J/kg]

### 熱・水分の移動係数

$K_{sat}$：飽和透水係数(liquid permeability)[m2/s]  

$\delta_p$：絶乾時の透湿率(moisture permeability)[kg/m Pa s]

$\lambda$：熱伝導率 [W/mK]

$D_w$：透水係数 

$D_p$：透湿率（湿気伝導率） 

In [1]:
module bentheimer_sandstone

# 空隙率
const psi = 0.23

# 材料密度
const row = 1479.25

# 比熱
const C = 750.0

include("./liquid_water.jl")

# 熱容量
get_crow( ;phi::Float64 ) = C * row + water_property.crow * phi
get_crow( cell ) = get_crow( phi=get_phi( cell ) )

# van-Genuchten用情報
include("./van_genuchten.jl")

BeSand_vG = van_Genuchten.vG_parameter( 
    10.0/98.0,
    2.0,
    1.0 - ( 1.0 / 2.0 ),
    0.5
)

### 水分特性 ###
# 含水率
get_phi( ;miu::Float64 ) = van_Genuchten.get_phi( vG = BeSand_vG, miu = miu, phimax = psi )
get_phi( cell ) = van_Genuchten.get_phi( vG = BeSand_vG, miu = cell.miu, phimax = psi )

# 含水率の水分化学ポテンシャル微分
get_dphi( ;miu::Float64 ) = van_Genuchten.get_dphi( vG = BeSand_vG, miu = miu, phimax = psi )
get_dphi( cell ) = get_dphi( miu = cell.miu )

# 含水率から水分化学ポテンシャルの算出
get_miu_by_phi( ;phi::Float64 ) = van_Genuchten.get_miu(vG = BeSand_vG, phi = phi, phimax = psi )
get_miu_by_phi( cell ) = get_miu_by_phi( phi = cell.phi )

### 移動特性 ###
# 熱伝導率
const lam = 1.2
get_lam( cell ) = lam

# 飽和時の透水係数
const Ksat = 2.0e-7

# 絶乾湿の透湿率
const Dp = 2.0E-10

# 透水係数
get_dw( ;miu::Float64 ) = Ksat * van_Genuchten.get_kl( vG = BeSand_vG, miu = miu )
get_dw( cell ) = get_dw( ;miu = cell.miu )

# 透湿率（湿気伝導率）
get_dp( ;phi::Float64 ) = Dp * ( 1.0 - ( phi / psi ) * 0.9 )
get_dp( cell ) = get_dp( phi = get_phi( cell ) )

end

Main.bentheimer_sandstone

### Example

In [2]:
mutable struct test_bentheimer
    temp::Float64
    miu::Float64
    rh::Float64
    pv::Float64
    phi::Float64
end

In [3]:
test_ben_hygro = test_bentheimer( 293.15, -100.0, 0.7, 1000.0, 0.1 )

test_bentheimer(293.15, -100.0, 0.7, 1000.0, 0.1)

In [4]:
bentheimer_sandstone.get_crow( test_ben_hygro )

1.2033412188466084e6

In [5]:
bentheimer_sandstone.get_phi( test_ben_hygro )

0.02243253636402062

In [6]:
bentheimer_sandstone.get_dphi( test_ben_hygro )

0.00022219143707850422

In [7]:
bentheimer_sandstone.get_miu_by_phi( test_ben_hygro )

-20.29806887366382

In [8]:
bentheimer_sandstone.get_dw( test_ben_hygro )

1.4197782071427953e-12

In [9]:
bentheimer_sandstone.get_dp( test_ben_hygro )

1.8244410197598387e-10

##############################
###     物性値の確認      ###
###     グラフの描画      ###
plt.figure(figsize = (15.0, 6))
plt.xscale("log")
#plt.yscale("log")
plt.grid(which="both")
plt.xlabel("-chemical potential[J/kg]", fontsize = 15)
plt.ylabel("phi[-]", fontsize = 15)

water = [ sample(i-1) for i in range(10000) ]
phi = [ get_phi ( water[i] ) for i in range (10000) ]
miu = [ i for i in range(10000) ]
plt.plot( miu, phi )
plt.show()
###########################